In [1]:
import pickle

import pandas as pd
import scipy.sparse
import scipy.sparse
from sklearn import preprocessing

from lightfm import LightFM

In [2]:
df = pd.read_parquet('/pio/scratch/1/recommender_systems/processed/amazon-clothes/5-core/train.parquet')
test_df = pd.read_parquet('/pio/scratch/1/recommender_systems/processed/amazon-clothes/5-core/test.parquet')
categories = pd.read_json('/pio/scratch/1/recommender_systems/interim/Amazon/meta_Clothing_Shoes_and_Jewelry_categories.json', lines=True)

In [3]:
interactions = scipy.sparse.load_npz('/pio/scratch/1/i313924/data/lightfm_data/5_core_interactions.npz')
test_interactions = scipy.sparse.load_npz('/pio/scratch/1/i313924/data/lightfm_data/5_core_test_interactions.npz')
dataset = pd.read_pickle('/pio/scratch/1/i313924/data/lightfm_data/5_core_dataset.pkl')

In [4]:
items = df['asin'].drop_duplicates()
item_mapping = dataset.mapping()[2]
items_categories = categories[categories.asin.isin(items)].drop_duplicates()
without_category = set(items.values) - set(items_categories.asin)
le = preprocessing.LabelEncoder()
unique_categories = items_categories.category_1.unique()
le.fit(unique_categories)
items_categories.category_1 = le.transform(items_categories.category_1)
for item in without_category:
    items_categories.loc[item_mapping[item]] = pd.Series({'category_1': -1, 'asin': item})
items_categories.index = items_categories.asin.map(lambda x: item_mapping[x])
items_categories = items_categories.sort_index()
item_category_mapping = items_categories.category_1

In [8]:
word2vec_embeddings = pd.read_parquet('/pio/scratch/1/i313924/data/word2vec/word2vec_240_embeddings.parquet')

In [10]:
word2vec_embeddings = word2vec_embeddings.rename(index=item_mapping)
word2vec_embeddings = word2vec_embeddings.sort_index()

In [11]:
model = LightFM(
    no_components=240,
    loss='warp',
    learning_schedule='adadelta',
    epsilon=2.45e-07,
    rho=0.958,
    # item_alpha=5.97e-05,
    user_alpha=2.06e-06,
    max_sampled=9,
    num_categories=len(unique_categories),
    item_category_mapping=item_category_mapping
)

my new implementation


In [ ]:
num_epochs = [5, 25, 50, 250, 500, 1000]
remaining = [num_epochs[0]] + [num_epochs[i] - num_epochs[i-1] for i in range(1, len(num_epochs))]
for i, epochs in enumerate(num_epochs):
    model.fit_partial(interactions, verbose=True, epochs=remaining[i], num_threads=8, word2vec_embeddings=word2vec_embeddings.values)
    pickle.dump(model, open(f'/pio/scratch/1/i313924/data/lightfm_data/model_{epochs}_word2vec_optimal_hyperparams_all_clusters_alpha_0.pkl', 'wb'), protocol=4)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

category a:  2 category_b:  7


Epoch:  20%|██        | 1/5 [00:28<01:52, 28.23s/it]

category a:  7 category_b:  5


Epoch:  40%|████      | 2/5 [00:51<01:16, 25.39s/it]

...

category a:  2 category_b:  5


Epoch:  68%|██████▊   | 137/200 [25:37<11:38, 11.08s/it]

category a:  2 category_b:  3


Epoch:  69%|██████▉   | 138/200 [25:48<11:27, 11.10s/it]

category a:  6 category_b:  9
